In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
from bs4 import BeautifulSoup
import urllib2
import requests

In [21]:
import json
with open("polinfo.json") as json_file:
    polinfo = json.load(json_file)
    json_file.close()

In [ ]:
polinfo.keys()[0:2]

In [367]:
#polinfo['candidate.php?candidate_id=4063']

In [ ]:
#soup=BeautifulSoup(polinfo['candidate.php?candidate_id=4063'],"html.parser")
#print soup.prettify()

In [64]:
def movable_assets(info,url):
    soup=BeautifulSoup(info[url],"html.parser")
    table = soup.find("a", attrs={"name": "movable_assets"}).find("table")
    madict={}
    madict["url"]="http://myneta.info/ls2014/"+str(url)
    for r in table.findAll('td'):
        try:
            if r.get_text()=="i":
                madict["Cash"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="ii":
                madict["Deposits in Banks, Financial Institutions and Non-Banking Financial Companies"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="iii":
                madict["Bonds, Debentures and Shares in companies"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")
            
            #this row is tricky: use different tag and attribute to pick up text. Limit specificies # of iteration. for findall method
            if r.get_text()=="iv":
                subtypes= r.findAllNext('span', attrs={"style":"color:black"},limit=2)
                sublist=[x.get_text().strip().encode('ascii', 'ignore').replace("Rs","") for x in subtypes]
                madict["NSS, Postal Savings etc"] = sublist[0]
                madict["LIC or other insurance Policies"] = sublist[1]

            if r.get_text()=="v":
                madict["Personal loans/advance given"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="vi":
                madict["Motor Vehicles (details of make, etc.)"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="vii":
                madict["Jewellery (give details weight value)"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="viii":
                madict["Other assets, such as values of claims / interests"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="Gross Total Value (as per Affidavit)":
                madict["Gross Total Value (as per Affidavit)"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="Totals (Calculated as Sum of Values)":
                madict["Totals (Calculated as Sum of Values)"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

        except:
            pass              

    return madict

In [65]:
def immovable_assets(info,url):
    soup=BeautifulSoup(info[url],"html.parser")
    table = soup.findAll("table",attrs={"class":"table1"})[2]
    immadict={}
    immadict["url"]="http://myneta.info/ls2014/"+str(url)
    for r in table.findAll('td'):
        try:
            if r.get_text()=="i":
                immadict["Agricultural Land"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="ii":
                immadict["Non Agricultural Land"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="iii":
                immadict["Commercial Buildings"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")
            
            if r.get_text()=="iv":
                immadict["Residential Buildings"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="v":
                immadict["Others"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")
            
            if r.get_text()=="Total Current Market Value of (i) to (v) (as per Affidavit)":
                immadict["Total Current Market Value of (i) to (v) (as per Affidavit)"] = r.findNext('span', attrs={"style":"color:green"}).get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="Totals Calculated":
                immadict["Totals Calculated"] = r.findNext('span', attrs={"style":"color:purple"}).get_text().strip().encode('ascii', 'ignore').replace("Rs","")

                
        except:
            pass
    
    return immadict
                

In [66]:
def cleaner(d):
    for key in d.keys():
        if d[key]=='Nil':
            d[key]==np.nan
        else:
            try:
                d[key]=float(d[key].replace(',',''))
            except:
                pass
    return d

In [67]:
%%time
movassets={}
for url in polinfo.keys():
    try:
        massetdict=cleaner(movable_assets(polinfo,url))
        movassets[url] = massetdict
        
    except TypeError: 
        print "error: ",url
        

error:  candidate.php?candidate_id=6112
error:  candidate.php?candidate_id=2923
error:  candidate.php?candidate_id=8068
error:  candidate.php?candidate_id=1469
error:  candidate.php?candidate_id=5058
error:  candidate.php?candidate_id=5390
error:  candidate.php?candidate_id=4001
CPU times: user 18min 30s, sys: 12.9 s, total: 18min 43s
Wall time: 21min 59s


In [13]:
#fd=open("movassets.json","w")
#json.dump(movassets, fd)
#fd.close()

In [39]:
%%time
immovassets={}
for url in polinfo.keys():
    try:
        immassetdict=cleaner(immovable_assets(polinfo,url))
        immovassets[url] = immassetdict
        
    except TypeError: 
        print "error: ",url
        

error:  candidate.php?candidate_id=6112
error:  candidate.php?candidate_id=2923
error:  candidate.php?candidate_id=8068
error:  candidate.php?candidate_id=1469
error:  candidate.php?candidate_id=5058
error:  candidate.php?candidate_id=5390
error:  candidate.php?candidate_id=4001
CPU times: user 19min 35s, sys: 13.1 s, total: 19min 48s
Wall time: 21min 18s


In [55]:
len(movassets)

8197

In [115]:
movassets=pd.DataFrame(movassets)
movassets=movassets.transpose()

In [116]:
movassets=movassets.replace('Nil',np.nan)
movassets.head(2)

,"Bonds, Debentures and Shares in companies",Cash,"Deposits in Banks, Financial Institutions and Non-Banking Financial Companies",Gross Total Value (as per Affidavit),Jewellery (give details weight value),LIC or other insurance Policies,"Motor Vehicles (details of make, etc.)","NSS, Postal Savings etc","Other assets, such as values of claims / interests",Personal loans/advance given,Totals (Calculated as Sum of Values),url
candidate.php?candidate_id=10,NaN,500,250,750,NaN,NaN,NaN,NaN,NaN,NaN,750,http://myneta.info/ls2014/candidate.php?candid...
candidate.php?candidate_id=100,20000000,14250000,180914202,373216000,21150000,NaN,31086774,2200000,NaN,103700000,368684524,http://myneta.info/ls2014/candidate.php?candid...


In [117]:
movassets.to_csv("movassets.csv", header=True, index=True, encoding='utf-8')

In [118]:
immovassets=pd.DataFrame(immovassets)
immovassets=immovassets.transpose()

In [119]:
immovassets=immovassets.replace('Nil',np.nan)
immovassets.head(2)

,Agricultural Land,Commercial Buildings,Non Agricultural Land,Others,Residential Buildings,Total Current Market Value of (i) to (v) (as per Affidavit),Totals Calculated,url
candidate.php?candidate_id=10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://myneta.info/ls2014/candidate.php?candid...
candidate.php?candidate_id=100,NaN,120000000,9000000,NaN,628000000,757000000,757000000,http://myneta.info/ls2014/candidate.php?candid...


In [120]:
immovassets.to_csv("immovassets.csv", header=True, index=True, encoding='utf-8')

In [402]:
#example:
dft=pd.DataFrame(massets[polinfo.keys()[0]])
dft.set_index('srn',inplace=True)
dft.head()

,dependent1,dependent2,dependent3,description,self,spouse,total
srn,,,,,,,
[i],[Nil],[Nil],[Nil],[Cash],"[ 10,000 , 10 Thou+, ]","[ 10,000 , 10 Thou+, ]","[ Rs 20,000 , 20 Thou+]"
[ii],[Nil],[Nil],[Nil],"[Deposits in Banks, Financial Institutions and...","[Current A/c , 3,50,000 , 3 Lacs+, ]","[Saving A/c , 75,000 , 75 Thou+, ]","[ Rs 4,25,000 , 4 Lacs+]"
[iii],[Nil],[Nil],[Nil],"[Bonds, Debentures and Shares in companies]",[Nil],[Nil],[ Nil ]
[iv],[Nil],[Nil],[Nil],"[(a), NSS, Postal Savings etc]",[Nil],[Nil],[ Nil ]
-,[Nil],[Nil],[Nil],"[(b), LIC or other insurance Policies , <br><s...","[HDFC Life Insurance, 2,50,000 , 2 Lacs+, ]","[LIC Policy, 1,95,000 , 1 Lacs+, , HDFC Li...","[ Rs 6,95,000 , 6 Lacs+]"


In [112]:
import gc
gc.collect()

0